The Cells from `Connecting the GPU` to `Installing VSGAN` are required. They must also be run in the shown order.

Yuuno is a convenience IPython module but it is not required to be used. You may simply use vspipe instead.

In [1]:
#@title Connect to GPU
#@markdown Enable GPU in Runtime -> Change runtime type first
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print("GPU:", gpu[1] or "None")
print("The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.")
print("Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.")

GPU: Tesla P100-PCIE-16GB
The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.
Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.


In [ ]:
#@title Install apt-fast for parallel apt downloads
!bash -c "$(curl -sL https://git.io/vokNn)"
print("apt-fast installed")

In [ ]:
#@title Install VapourSynth
#@markdown This does not install ANY scripts or plugins, just VapourSynth.
sources = !cat /etc/apt/sources.list
if "deb https://www.deb-multimedia.org" not in sources:
    with open("/etc/apt/sources.list", "a+") as f:
        f.write("deb https://www.deb-multimedia.org buster main non-free\n")
if "ftp.us.debian.org/debian/ sid main" not in sources:
    with open("/etc/apt/sources.list", "a+") as f:
        f.write("deb http://ftp.us.debian.org/debian/ buster main\n")
!apt-fast update -oAcquire::AllowInsecureRepositories=true
!apt-fast install -y --allow-unauthenticated deb-multimedia-keyring
print("Prepared apt sources!")
!apt-fast install -y --allow-unauthenticated vapoursynth
print("Installed VapourSynth")

In [ ]:
#@title Install VSGAN
!pip install vsgan

In [ ]:
#@title Mount Google Drive to /gdrive

import os
from google.colab import drive

if not os.path.isdir("/drive"):
  drive.mount("/gdrive", force_remount=True)

In [ ]:
#@title Install x264
!apt-fast install -y --allow-unauthenticated x264

In [ ]:
#@title Install Yuuno
#@markdown This is an optional convenience feature, allowing IPython % magic commands.
!pip install yuuno
!yuuno jupyter install

In [ ]:
#@title After installing Yuuno, reboot (NOT RESET!) the current instance then start this.
#@markdown For some reason it thinks yuuno isn't an IPython module if you don't first restart.
%load_ext yuuno

In [ ]:
#@title Start an example encode and download it
%%vspipe --y4m | x264 - --demuxer y4m -o example.mkv
clip = core.std.BlankClip(color=[0,255,255])
clip = core.resize.Spline36(clip, format=vs.YUV420P8, matrix=1)
clip.set_output()

EncodeWidget(length=240, start_time=1621402951)

In [ ]:
#@title Download the example encode
from google.colab import files
files.download("example.mkv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Example Yuuno vspreview call.
#@markdown Note: Yuuno features that return a preview dialog doesn't seem to work on Google Colab.
%%vspreview
clip = core.std.BlankClip(color=[0,255,255])
clip.set_output()

Preview(clip=<vapoursynth.VideoNode object at 0x7f3ff21179b0>)